# Movies Recommend System

レビュー類似度を用いた Movies Recommend System の開発

使用する映画レビューサイトは　http://www.jtnews.jp/index.html

### 構成

1. スクレイピング/SQL
2. レビューの形態素解析
3. 特徴量行列の作成
4. モデル作成

### 準備

In [1]:
#パンダスの準備
import pandas as pd
from pandas import Series,DataFrame

In [2]:
#SQLの準備
import sqlite3
con = sqlite3.connect("movies.db")

In [3]:
# 関数を作って、SQL文の実行結果をDataFrameにして返します。
def sql_to_df(sql_query):

    # read_sqlの引数に、SQL文とデータベースへのConnectionを渡します。
    df = pd.read_sql(sql_query, con)

    # 結果のDataFrameを返します。
    return df

In [6]:
import MeCab
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

### 1. スクレイピング/SQL

In [4]:
watch = '''
青春映画がみたい。高校生が地球を守るために奮闘する話とか。
ジブリのコクリコ坂とか、単純な青春恋愛物語的な要素があるものでもOK.
見たあと気持ちが爽やかになる映画。
'''

In [ ]:
#con.close

### 2. レビューの形態素解析

In [34]:
text = watch
mecab.parse('')#文字列がGCされるのを防ぐ
node = mecab.parseToNode(text)
i_id=0
while node:
    #単語を取得
    word = node.surface
    #品詞を取得
    pos = node.feature.split(",")[1]
    #print('{}'.format(word))
    #print('{0} , {1}'.format(word, pos))
        
    #次の単語に進める
    node = node.next
    
    sql_query = '''
    INSERT INTO watch_cnt_table(id, word, num) values({i_id}, "{word}", 1)
    '''.format(i_id=i_id, word=word)
        
    con.execute(sql_query)
    
i_id += 1


青春映画
が
みたい
。
高校生
が
地球
を
守る
ため
に
奮闘
する
話
とか
。
ジブリ
の
コクリコ坂
とか
、
単純
な
青春
恋愛
物語
的
な
要素
が
ある
もの
で
も
OK
.
見
た
あと
気持ち
が
爽やか
に
なる
映画
。



##### 事前に作成しているテーブル

CREATE TABLE cnt_table(
    id int,
    title char(50),
    word char(30),
    num int
);

In [35]:
sql_query = '''
SELECT * FROM watch_cnt_table
'''
words_df = sql_to_df(sql_query)

In [36]:
words_df.tail()

,id,word,num
43,0,に,1
44,0,なる,1
45,0,映画,1
46,0,。,1
47,0,,1


In [14]:
sql_query = '''
DELETE  FROM watch_cnt_table
'''
con.execute(sql_query)

In [ ]:
#con.close()

### 3. 特徴量行列の作成

In [10]:
i = 1
words_cnt = []

In [11]:
#The first step —————————————————————————————————————————————————
#全映画で使われたwordをとりだす。
sql_query = '''
SELECT word FROM cnt_table WHERE word not like '%''%' GROUP BY word ORDER BY sum(num) desc
'''
words_used_df = sql_to_df(sql_query)

In [12]:
words_used_df.tail(1)

,word
31566,￣*


In [20]:
31566 / 1900

16.613684210526316

In [46]:
query_all = []

n = 0

for num in range(1,17):     
    query = '''SELECT '''
    for item in range(len(words_used_df)):
        if n <= num*1900:
            #レビューIDごとに出現しているwordでカウントする。
            query += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{n}, '''.format(word=words_used_df.iloc[item, 0], n=n)
        
            n += 1
    
            if n == num*1900:
                break
    
    query += '''id FROM watch_cnt_table GROUP BY id'''
    #print(query)
    query_all.append(query)

In [47]:
query = '''
SELECT distinct id FROM watch_cnt_table
'''
id_df = sql_to_df(query)

In [48]:
for num in range(1,17):
    print(num)
    res_df = sql_to_df(query_all[num-1])
    if num == 1:
        all_df = pd.merge(id_df, res_df, on='id')
    elif num > 1:
        all_df = pd.merge(all_df, res_df, on='id')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [56]:
all_df = all_df.drop('id', axis=1)

In [57]:
all_df

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a30390,a30391,a30392,a30393,a30394,a30395,a30396,a30397,a30398,a30399
0,1,3,1,4,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
con.close

<function Connection.close>

### 4. モデル作成

In [69]:
res_df = pd.read_csv('tokuchouryou.csv', index_col=0 )

In [70]:
res_df

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a30391,a30392,a30393,a30394,a30395,a30396,a30397,a30398,a30399,title
0,3,4,6,2,2,3,4,3,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
1,5,4,5,4,3,3,3,3,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
2,5,6,5,6,4,3,4,1,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
3,3,1,2,1,3,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
4,4,5,2,3,5,2,2,3,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
5,3,0,1,3,1,0,4,1,2,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
6,3,3,3,3,3,0,6,1,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
7,3,2,2,3,4,0,2,1,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
8,3,3,1,3,4,1,2,0,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に
9,3,3,1,2,0,1,1,2,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に


ここでは、UdemyのPython実践講座の Lec80-81 を参考にしてロジスティック回帰での多値分類を行う。

In [71]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [72]:
from sklearn import linear_model

参考 : http://qiita.com/yshi12/items/9f3f8ae69588da9f018f

In [73]:
res_df.target = res_df['title']

In [74]:
res_numpyMatrix_target = res_df.target.as_matrix()

In [75]:
res_df.data = res_df.iloc[ : , 0:30400]

In [76]:
res_numpyMatrix_data = res_df.iloc[ : , 0:30400].as_matrix()

In [77]:
# 説明変数をXに
X = res_numpyMatrix_data

#目的変数をYに
Y = res_numpyMatrix_target

In [78]:
#pandas.DataFrameにしておきましょう。

_data = DataFrame(X)

_target = DataFrame(Y,columns=['title'])

scikit-learnを使った多クラス分類


In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

logreg = LogisticRegression()

# データを使って学習します。
logreg.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [86]:
from sklearn.externals import joblib
joblib.dump(logreg,"model_dir/model")

['model_dir/model',
 'model_dir/model_01.npy',
 'model_dir/model_02.npy',
 'model_dir/model_03.npy']

In [82]:
all_numpyMatrix_data = all_df.as_matrix()
X_test = all_numpyMatrix_data
_test_data = DataFrame(X_test)

In [83]:
# テストデータを予測します。
Y_pred = logreg.predict(X_test)

In [84]:
Y_pred

array(['Ｅｍｍａ／エマ(1996)'], dtype=object)